In [1]:
import numpy as np
import bct
from nilearn.connectome import vec_to_sym_matrix
import pdb

In [2]:
def my_degrees_wu(CIJ):
    '''
    Node degree is the sum of weights of links connected to the node.

    Parameters
    ----------
    CIJ : NxN np.ndarray
        undirected binary/weighted connection matrix

    Returns
    -------
    deg : Nx1 np.ndarray
        node degree

    Notes
    -----
    Weight information is discarded.
    '''
    #CIJ = binarize(CIJ, copy=True)  # ensure CIJ is binary
    return np.sum(CIJ, axis=0)

def my_threshold_proportional(W, p, copy=True):
    """
    use absolute values of weights to threshold the matrix
    because the weights are signed.
    """
    W2 = W.copy()
    W2 = np.abs(W2)
    
    if copy:
        W = W.copy()
    
    W2= bct.threshold_proportional(W2, p, copy)
    
    rows, cols = W2.shape
    
    for i in range(rows):
        for j in range(cols):
            if W2[i][j] ==0:
                W[i][j] = 0
    
    return W

In [15]:
def graph_metrics(x_origin, p, signed=True):
    """
    functionality: 
           calculate the graph metrics for an individual
    params:
           x_origin: n by n correlation matrix (original)
           p:        threshold x to keep only propotion of connections. 0<p<=1
    return:
           an ndarray that contains all the graph metrics (1 by m, where m is the number of new features)
    """
    if signed:
        x = my_threshold_proportional(x_origin, p, copy=True)
    else:
        x = bct.threshold_proportional(x_origin, p, copy=True)
    
    #community affiliation vector
    if signed:
        ci, _ = bct.modularity_louvain_und_sign(x)
    else:
        ci, _ = bct.modularity_und(x)
    
    
    metrics = np.array([])
    # 5 local network metrics
    ## 1). degree
    
    #degrees = bct.degrees_und(x)
    #s = my_degrees_wu(x)
    if signed:
        pos, neg = bct.gateway_coef_sign(x, ci, centrality_type='degree')
        degrees = pos + neg
    else:
        degrees = bct.degrees_und(x)
    
    metrics = np.concatenate((metrics, degrees))
    
    ## 2). participation coefficient
    ## participation_coef_sign.m (WU signed networks)
    
    if signed:
        pos, neg = bct.participation_coef_sign(x, ci)
        pc = pos + neg
    else:
        pc = bct.participation_coef(x,ci)
    
    metrics = np.concatenate((metrics, pc))
    
    ## 3).betweeness centrality
    if signed:
        pos, neg = bct.gateway_coef_sign(x, ci, centrality_type='betweenness')
        bc = pos + neg
    else:
        L = bct.weight_conversion(x, 'lengths', copy=True)
        bc = bct.betweenness_wei(L)
    
    metrics = np.concatenate((metrics, bc))
    
    ## 4).local efficiency
    ## 5).ratio of local to global efficiency
    if signed:
        # possible a problem because x should only have numbers from [0,1], if signed, will have negatives
        le = bct.efficiency_wei(x, 'local')
        ge = bct.efficiency_wei(x, 'global')
    else:
        le = bct.efficiency_wei(x, 'local')
        ge = bct.efficiency_wei(x, 'global')
    
    metrics = np.concatenate((metrics, le))
    metrics = np.concatenate((metrics, le/ge))
    
    # 4 global network metrics
    ## 1). average path length
    L = bct.weight_conversion(x, 'lengths', copy=True)
    apl = bct.charpath(L, include_diagonal=False, include_infinite=False)
    apl = apl[0]
    
    ## 2). average clustering coefficient
    if signed:
        pos, neg = bct.clustering_coef_wu_sign(x)
        acc = pos+neg
    else:
        acc = bct.clustering_coef_wu(x)
    
    acc = np.mean(acc)
    
    ## 3).global efficiency
    ge = ge
    
    ## 4). small-worldness
    seed = 42
    if signed:
        randomX, _ = bct.randmio_und_signed(x, 5, seed = seed)
        
        pos, neg = bct.clustering_coef_wu_sign(randomX)
        random_acc = pos+neg
    else:
        randomX, _ = bct.randmio_und(x, 5, seed=seed)
        random_acc = bct.clustering_coef_wu(randomX)
    random_acc = np.mean(random_acc)
    
    L = bct.weight_conversion(randomX, 'lengths', copy=True)
    random_apl = bct.charpath(L, include_diagonal=False, include_infinite=False)
    random_apl = random_apl[0]
    
    small_worldness = (acc/random_acc)/(apl/random_apl)
    
    metrics = np.concatenate((metrics, np.array([apl, acc, ge, small_worldness])))
    #pdb.set_trace()
    # the length of metrics is n*5+4, where n is the number of ROIs
    return metrics
    

In [4]:
X = np.genfromtxt('AAL_X_corr.csv', delimiter=',')
X2 = vec_to_sym_matrix(X)
print(X2.shape)
x = X2[0]

(871, 116, 116)


In [ ]:
graph_metrics(x, 0.6)

In [16]:
bct.randmio_und??

In [21]:
bct.efficiency_bin??

6786.0

In [ ]:
def my_threshold_proportional(W, p, copy=True):
    '''
    This function "thresholds" the connectivity matrix by preserving a
    proportion p (0<p<1) of the strongest weights (based on absolute values). All other weights, and
    all weights on the main diagonal (self-self connections) are set to 0.

    If copy is not set, this function will *modify W in place.*

    Parameters
    ----------
    W : np.ndarray
        weighted connectivity matrix
    p : float
        proportional weight threshold (0<p<1)
    copy : bool
        if True, returns a copy of the matrix. Otherwise, modifies the matrix
        in place. Default value=True.

    Returns
    -------
    W : np.ndarray
        thresholded connectivity matrix

    Notes
    -----
    The proportion of elements set to 0 is a fraction of all elements
    in the matrix, whether or not they are already 0. That is, this function
    has the following behavior:

    >> x = np.random.random_sample((10,10))
    >> x_25 = threshold_proportional(x, .25)
    >> np.size(np.where(x_25)) #note this double counts each nonzero element
    46
    >> x_125 = threshold_proportional(x, .125)
    >> np.size(np.where(x_125))
    22
    >> x_test = threshold_proportional(x_25, .5)
    >> np.size(np.where(x_test))
    46

    That is, the 50% thresholding of x_25 does nothing because >=50% of the
    elements in x_25 are aleady <=0. This behavior is the same as in BCT. Be
    careful with matrices that are both signed and sparse.
    '''
    from bct import teachers_round as round

    if p > 1 or p < 0:
        raise BCTParamError('Threshold must be in range [0,1]')
    if copy:
        W = W.copy()
    # ning 2023/12/03
    W2 = W.copy()
    W2 = np.abs(W2)
    
    n = len(W)                                          # number of nodes
    np.fill_diagonal(W2, 0)                      # clear diagonal
    np.fill_diagonal(W, 0) 
    
    if np.allclose(W, W.T):                             # if symmetric matrix
        W2[np.tril_indices(n)] = 0               # ensure symmetry is preserved
        W[np.tril_indices(n)] = 0 
        ud = 2                                          # halve number of removed links
    else:
        ud = 1

    ind = np.where(W2)                                   # find all links
    pdb.set_trace()
    #I = np.argsort(W[ind])[::-1]                # sort indices by magnitude
    I = np.argsort(W2[ind])[::-1]                # sort indices by absolute magnitude
    
    en = int(round((n * n - n) * p / ud))               # number of links to be preserved

    W[(ind[0][I][en:], ind[1][I][en:])] = 0  # apply threshold
    #W[np.ix_(ind[0][I][en:], ind[1][I][en:])]=0

    if ud == 2:                                         # if symmetric matrix
        W[:, :] = W + W.T                                               # reconstruct symmetry

    return W